In [161]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers import Dense,Input
from keras.models import Model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

In [5]:
X_train,X_test,y_train,y_test = train_test_split(data,target,shuffle = True,test_size = 0.3,random_state = 1)

In [6]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [420]:
def models(layers,nn_in_l,act = 'relu'):
    inputs = Input(shape = (13,))
    x1 = Dense(nn_in_l[0],activation = act)(inputs)
    for i in range(1,layers):
        x1 = Dense(nn_in_l[i],activation = act)(x1)
    return Model(inputs =inputs,outputs = x1)
    

In [145]:
%%time
RMSE = []
model4 = models(4,[100,2000,100,1])
model4.compile(optimizer = 'Adam',loss = 'MSE')
model4.fit(X_train,y_train,epochs = 20,verbose = 0)

Wall time: 2.47 s


In [146]:
np.sqrt(model4.evaluate(X_test,y_test,verbose = 0))

3.215551472259767

In [49]:
model5 = models(5,[64,1000,1000,32,1])
model5.compile(optimizer = 'Adam',loss = 'MSE')
model5.fit(X_train,y_train,epochs = 20,verbose = 0)

In [405]:
RMSE.append(np.sqrt(model5.evaluate(X_test,y_test,verbose = 0)))
np.sqrt(model5.evaluate(X_test,y_test,verbose = 0))

2.855858647019381

In [51]:
model6 = models(6,[128,64,1000,1000,16,1])
model6.compile(optimizer = 'Adam',loss = 'MSE')
model6.fit(X_train,y_train,epochs = 20,verbose = 0)

In [406]:
RMSE.append(np.sqrt(model6.evaluate(X_test,y_test,verbose = 0)))
np.sqrt(model6.evaluate(X_test,y_test,verbose = 0))

3.0514709240151485

In [56]:
model7 = models(7,[64,128,64,1000,16,8,1])
model7.compile(optimizer = 'Adam',loss = 'MSE')
model7.fit(X_train,y_train,epochs = 20,verbose = 0)

In [57]:
np.sqrt(model7.evaluate(X_test,y_test,verbose = 0))

3.1167707810036913

In [60]:
model8 = models(8,[64,128,64,1000,1000,8,4,1])
model8.compile(optimizer = 'Adam',loss = 'MSE')
model8.fit(X_train,y_train,epochs = 20,verbose = 0)

In [407]:
RMSE.append(np.sqrt(model8.evaluate(X_test,y_test,verbose = 0)))
np.sqrt(model8.evaluate(X_test,y_test,verbose = 0))

2.7070306334832

In [78]:
model9 = models(9,[64,128,256,512,1024,512,256,64,1])
model9.compile(optimizer = 'Adam',loss = 'MSE')
model9.fit(X_train,y_train,epochs = 20,verbose = 0)

In [408]:
RMSE.append(np.sqrt(model9.evaluate(X_test,y_test,verbose = 0)))
np.sqrt(model9.evaluate(X_test,y_test,verbose = 0))

2.7786444128021546

In [109]:
%%time
model10 = models(10,[64,128,256,512,1024,1024,64,32,64,1])
model10.compile(optimizer = 'Adam',loss = 'MSE')
model10.fit(X_train,y_train,epochs = 20,verbose = 0)

Wall time: 10.5 s


In [110]:
np.sqrt(model10.evaluate(X_test,y_test,verbose = 0))

3.1160074165755463

In [141]:
%%time
model11 = models(11,[64,128,256,512,1024,2048,1024,64,32,64,1])
model11.compile(optimizer = 'Adam',loss = 'MSE')
model11.fit(X_train,y_train,epochs = 20,verbose = 0)

Wall time: 26.8 s


In [142]:
np.sqrt(model11.evaluate(X_test,y_test,verbose = 0))

3.488686259339047

In [139]:
%%time
model12 = models(12,[64,128,256,512,1024,1024,2048,1024,64,64,64,1])
model12.compile(optimizer = 'Adam',loss = 'MSE')
model12.fit(X_train,y_train,epochs = 20,verbose = 0)

Wall time: 34.9 s


In [140]:
np.sqrt(model12.evaluate(X_test,y_test,verbose = 0))

3.6170088721839653

**Наилучшие метрики получались когда модель постепенно "толстела" к центральным слоям**

In [460]:
def expr(optimizers = [legacy.RMSprop(),legacy.Adagrad()],act = 'relu',RMSE_DATA = pd.DataFrame()):
    for opt in optimizers:
        RMSE = []
        for neu in nn:
            model = models(len(neu), neu)
            model.compile(optimizer=opt,loss = 'MSE')
            model.fit(X_train, y_train, epochs=20,verbose = 0)
            rmse = np.sqrt(model.evaluate(X_test, y_test, verbose=0))
            RMSE.append(rmse)
        temp = pd.DataFrame(RMSE).T
        temp['optimizer'] = opt.get_config()['name']
        temp['activation_func'] = act.name
        temp.columns = RMSE_DATA.columns
        RMSE_DATA = pd.concat([RMSE_DATA,temp],axis = 0)
    return RMSE_DATA

In [454]:
nn_5 = [64,1000,1000,32,1]
nn_6 = [128,64,1000,1000,16,1]
nn_8 =  [64,128,64,1000,1000,8,4,1]
nn_9 = [64,128,256,512,1024,512,256,64,1]
nn = [nn_5,nn_6,nn_8,nn_9]

**Наилучшие конфигурации числа нейронов в слоях**

**Замечание:Чем больше слоев и нейронов, тем чувствительней модель к начальым весам**

In [455]:
from tensorflow.keras.optimizers import *

In [465]:
RMSE_DATA = pd.DataFrame(np.array(RMSE))
RMSE_DATA = RMSE_DATA.T
RMSE_DATA.columns = ['5_neurons','6_neurons','8_neurons','9_neurons']
RMSE_DATA['optimizer'] = 'Adam'
RMSE_DATA['activation_func'] = 're_lu'
columns = RMSE_DATA.columns
RMSE_DATA

,5_neurons,6_neurons,8_neurons,9_neurons,optimizer,activation_func
0,2.855859,3.051471,2.707031,2.778644,Adam,re_lu


In [479]:
from keras.layers import ReLU,LeakyReLU,ELU

In [467]:
data = expr(optimizers = [legacy.RMSprop(),legacy.Adagrad()],act = ReLU(),RMSE_DATA = RMSE_DATA) 

In [469]:
data = expr(optimizers = [legacy.RMSprop(),legacy.Adagrad()],act = LeakyReLU(),RMSE_DATA = data) 

In [471]:
data = expr(optimizers = [legacy.Adam(),],act = LeakyReLU(),RMSE_DATA = data) 

In [475]:

data = expr(optimizers = [legacy.Adam(),legacy.RMSprop(),legacy.Adagrad()],act = ELU(),RMSE_DATA = data) 

In [476]:
data

,5_neurons,6_neurons,8_neurons,9_neurons,optimizer,activation_func
0,2.855859,3.051471,2.707031,2.778644,Adam,re_lu
0,2.991158,3.993815,3.272202,5.752151,RMSprop,re_lu_1
0,4.654756,4.169523,4.795677,4.000703,Adagrad,re_lu_1
0,4.504513,3.357481,3.258912,3.207686,RMSprop,leaky_re_lu_26
0,3.951305,4.184193,3.917098,3.550106,Adagrad,leaky_re_lu_26
0,3.110889,2.927111,2.950021,2.858373,Adam,leaky_re_lu_27
0,3.041395,2.835903,3.068053,2.919495,Adam,elu
0,2.727041,5.389580,24.896540,4.206643,RMSprop,elu
0,4.574432,4.483676,5.097102,3.703732,Adagrad,elu


In [ ]:
data = expr(optimizers = [legacy.Adam(),legacy.RMSprop(),legacy.Adagrad()],act = Selu(),RMSE_DATA = data) 

**Наилучшие метрики показывет Adam.**

**Наилучшая конфигарация модели это 8 слоев,[64,128,64,1000,1000,8,4,1] - нейронов в каждом слое, с оптимизатором Adam и с функцией активации ReLU.**

**Вторая лучшая конфиграция это 5-ти слойная сеть,[64,1000,1000,32,1] - нейронов в каждом слое, с оптимизатором RMSprop и с функцией активации ELU**

***(((Для чего можно использовать tensorflow, какие более грубокие настройки сети я могу с его помощью делать, помимо наблюдения за градиентами???)))***